In [44]:
import os
import numpy as np
import random

In [9]:
tile_dir = '../data/npz_all/npz/tile/xla/'
train_dir = tile_dir + 'train/'
valid_dir = tile_dir + 'valid/'
test_dir = tile_dir + 'test/'



print(len(os.listdir(train_dir)))
print(len(os.listdir(valid_dir)))
print(len(os.listdir(test_dir)))

5709
676
844


In [11]:
print(os.listdir(_dir)[0])

3ad7a21fdfc9fce79d2b9c3f83397989.npz


In [19]:
data = np.load(valid_dir + os.listdir(valid_dir)[0])

In [20]:
for k, v in data.items():
    print(k, v.shape)

node_feat (27, 140)
node_opcode (27,)
edge_index (26, 2)
config_feat (1893, 24)
config_runtime (1893,)
config_runtime_normalizers (1893,)


In [22]:
np.mean(data['config_runtime'])

1760332.9302694136

In [55]:
def speed_score(y_true, y_pred, k):
    y_fastest = np.min(y_true)

    slowdowns_actual = y_true / y_fastest

    top_k_indices = y_pred[:k]
    
    top_k_predicted_times = np.min(y_true[top_k_indices])

    ratio = top_k_predicted_times / y_fastest

    return 2 - ratio 

print(speed_score(np.array([1.24, 0.231, 4.2, 2.01]), np.array([2, 0, 3, 1]), 1))

-16.181818181818183


In [57]:
def predict(model, filename):
    data = np.load(filename)

    runtimes = data['config_runtime'] / data['config_runtime_normalizers']

    return runtimes,model(data)

def random_model(data):
    indices = list(range(len(data['config_runtime'])))
    random.shuffle(indices)
    return indices

filenames = [valid_dir + f for f in os.listdir(valid_dir)]

for i in range(10):
    scores = []
    for f in filenames:
        runtimes, preds = predict(random_model, f)
    
        scores.append(speed_score(runtimes, preds, 3))
    
    
    print(np.mean(scores))

0.20588285240273144
0.11013976804938272
0.07634861388709904
0.08908316248150643
0.10548635153856464
0.10646708337931983
0.054634522607414165
0.13785412973072972
0.16659947780720924
0.10382217359189513
